# Tutorial 3: run CANAL on cross-tissue experiments

In [1]:
import os
os.chdir('/data/wanh/CANAL/') 
import sys
sys.path.append('/data/wanh/CANAL/')
import argparse
from model import *
import time
import resource

## construct the CANAL model

In [2]:
experiments = 'Cross_tissue'
seed = 1
with open(f'./data/{experiments}/{experiments}_highly_gene_idx.csv') as csvfile:
      spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
      highly_variable_idx = []
      for row in spamreader:
            highly_variable_idx.append(row[0])

highly_variable_idx = [int(i) for i in highly_variable_idx]
highly_variable_idx = np.array(highly_variable_idx)

In [3]:
CANAL = CANAL_model(gpu_option = '1')

## stage 1

### load the pre-processed dataset ''Lung'' 

In [4]:
dataset_stage_1 = "TabulaMuris_Lung_10X"
data_path_stage_1 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_1)
adata_stage_1 = sc.read_h5ad(data_path_stage_1)
cell_type_stage_1 = adata_stage_1.obs['cell_type1']
print(adata_stage_1) 
print(np.unique(np.array(cell_type_stage_1), return_counts=True))

AnnData object with n_obs × n_vars = 4122 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['B cell', 'T cell', 'endothelial cell', 'macrophage',
       'natural killer cell', 'stromal cell'], dtype=object), array([ 187,  212,  412,  314,  721, 2276]))


### fine-tune the CANAL model for the first stage

In [5]:
CANAL.train(experiments = experiments, pre_dataset = "None", dataset = dataset_stage_1, 
            adata = adata_stage_1, cell_type = cell_type_stage_1, current_stage = 1, 
            is_final_stage = False, ckpt_dir = './ckpts/', rehearsal_size = 2000, 
            highly_variable_idx=highly_variable_idx, SEED = seed)

current data: AnnData object with n_obs × n_vars = 4122 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['B cell', 'T cell', 'endothelial cell', 'macrophage',
       'natural killer cell', 'stromal cell'], dtype=object), array([ 187,  212,  412,  314,  721, 2276]))
model constructing begin!



/data/wanh/CANAL/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


model constructing finished!

label train: [0 1 2 3 4 5] 6
label val: [0 1 2 3 4 5] 6
  ==  Begin finetuning: | Initial stage | Current stage: 1 | CANAL | Dataset: Cross_tissue TabulaMuris_Lung_10X ==
    ==  Epoch: 1 | Classification Loss: 1.796210 | Representation DL Loss: 0.000000  | Accuracy: 7.8419%  ==
    ==  Epoch: 1 | Validation CLS Loss: 1.795722 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.021864  == | Current accuracy: 0.070197  ==
[[  0   0   0  31   0   0]
 [  0   0   0  38   0   0]
 [  0   0   0  70   0   0]
 [  0   0   0  57   0   0]
 [  0   0   0 163   0   0]
 [  0   0   0 453   0   0]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.988520 | Representation DL Loss: 0.000000  | Accuracy: 71.1246%  ==
    ==  Epoch: 2 | Validation CLS Loss: 2.602915 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.860588  == | Current accuracy: 0.950739  ==
[[ 34   0   0   3   2   0]
 [  2  11   6   0  22   0]
 [  0   0  73   0   0   2]
 [  0   0   0

    ==  Epoch: 19 | Classification Loss: 0.424822 | Representation DL Loss: 0.000000  | Accuracy: 100.0000%  ==
    ==  Epoch: 19 | Validation CLS Loss: 2.915099 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.995456  == | Current accuracy: 0.997537  ==
[[ 39   0   0   0   0   0]
 [  0  45   0   0   0   0]
 [  0   0  84   0   0   0]
 [  0   0   0  60   0   0]
 [  0   0   0   0 141   0]
 [  2   0   0   0   0 441]]
Patience: 6 / 10
    ==  Epoch: 20 | Classification Loss: 0.425196 | Representation DL Loss: 0.000000  | Accuracy: 100.0000%  ==
    ==  Epoch: 20 | Validation CLS Loss: 2.823490 | Validation Representation DL Loss: 0.000000 | F1 Score: 0.992012  == | Current accuracy: 0.995074  ==
[[ 42   0   0   0   0   0]
 [  0  35   0   0   0   0]
 [  0   0  75   0   0   0]
 [  0   0   0  63   0   0]
 [  0   0   1   0 142   0]
 [  3   0   0   0   0 451]]
Patience: 7 / 10
    ==  Epoch: 21 | Classification Loss: 0.424549 | Representation DL Loss: 0.000000  | Accuracy: 100.0000% 

/data/wanh/CANAL/model.py:105: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:

AnnData object with n_obs × n_vars = 1712 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
cell type composition of this example bank:

(array(['B cell', 'T cell', 'endothelial cell', 'macrophage',
       'natural killer cell', 'stromal cell'], dtype=object), array([187, 212, 333, 314, 333, 333]))
dataset composition from each stage of this example bank:

(array([1]), array([1712]))




## stage 2

### load the pre-processed dataset ''Mammary_Gland'' 

In [5]:
dataset_stage_2 = "TabulaMuris_Mammary_Gland_10X"
data_path_stage_2 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_2)
adata_stage_2 = sc.read_h5ad(data_path_stage_2)
cell_type_stage_2 = adata_stage_2.obs['cell_type1']
print(adata_stage_2)  
print(np.unique(np.array(cell_type_stage_2), return_counts=True))

AnnData object with n_obs × n_vars = 3981 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['B cell', 'T cell', 'basal cell', 'endothelial cell',
       'luminal epithelial cell of mammary gland', 'macrophage',
       'stromal cell'], dtype=object), array([ 665, 1547,  341,  223,  411,  164,  630]))


### fine-tune the CANAL model for the second stage

In [7]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_1, dataset = dataset_stage_2, 
            adata = adata_stage_2, cell_type = cell_type_stage_2, current_stage = 2, 
            is_final_stage = False, ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: AnnData object with n_obs × n_vars = 3981 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['B cell', 'T cell', 'basal cell', 'endothelial cell',
       'luminal epithelial cell of mammary gland', 'macrophage',
       'stromal cell'], dtype=object), array([ 665, 1547,  341,  223,  411,  164,  630]))
model constructing begin!

new cell types: ['basal cell', 'luminal epithelial cell of mammary gland']
example bank for experience replay: AnnData object with n_obs × n_vars = 1712 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation'

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


    ==  Epoch: 1 | Classification Loss: 0.943613 | Representation DL Loss: 0.004290  | Accuracy: 83.0989%  ==
    ==  Epoch: 1 | Validation CLS Loss: 3.049186 | Validation Representation DL Loss: 0.008296 | F1 Score: 0.678923  == | Current accuracy: 0.837743  ==
[[160   0   0   0   0   0   0   0]
 [  6 338   0   0   3   0   0   0]
 [  0   0 118   0   0   0   0   0]
 [ 20   1   0  80   1   4   0   0]
 [  0   0   0   0  76   0   0   0]
 [  2   1   0   0   0 178   0   0]
 [  0   0   0   0   0  71   0   0]
 [  5   4   0   0   0  66   0   0]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.637936 | Representation DL Loss: 0.247472  | Accuracy: 95.6264%  ==
    ==  Epoch: 2 | Validation CLS Loss: 3.465677 | Validation Representation DL Loss: 0.275875 | F1 Score: 0.980252  == | Current accuracy: 0.984127  ==
[[191   4   0   0   0   0   0   0]
 [  0 343   0   0   4   0   0   0]
 [  0   0 101   0   0   0   0   0]
 [  2   2   0  99   0   0   0   0]
 [  0   0   0   0  56   0   0   0]
 

    ==  Epoch: 16 | Classification Loss: 0.475443 | Representation DL Loss: 0.164815  | Accuracy: 99.9560%  ==
    ==  Epoch: 16 | Validation CLS Loss: 3.531243 | Validation Representation DL Loss: 0.164516 | F1 Score: 0.997349  == | Current accuracy: 0.996473  ==
[[159   3   0   0   0   0   0   0]
 [  0 352   0   0   0   0   0   0]
 [  0   0 100   0   0   0   0   0]
 [  0   0   0 112   0   0   0   0]
 [  0   0   0   0  63   0   0   0]
 [  0   0   0   0   0 191   0   0]
 [  0   0   0   0   0   0  73   0]
 [  0   1   0   0   0   0   0  80]]
Patience: 9 / 10
    ==  Epoch: 17 | Classification Loss: 0.475245 | Representation DL Loss: 0.169262  | Accuracy: 99.9560%  ==
    ==  Epoch: 17 | Validation CLS Loss: 3.615410 | Validation Representation DL Loss: 0.154457 | F1 Score: 0.997205  == | Current accuracy: 0.997354  ==
[[151   1   0   0   0   0   0   0]
 [  0 337   0   0   1   0   0   0]
 [  0   0 123   0   0   0   0   0]
 [  0   0   0  93   0   0   0   0]
 [  0   0   0   0  65   0   0   

/data/wanh/CANAL/model.py:105: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:

AnnData object with n_obs × n_vars = 995 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
cell type composition of this example bank:

(array(['B cell', 'T cell', 'basal cell', 'endothelial cell',
       'luminal epithelial cell of mammary gland', 'macrophage',
       'natural killer cell', 'stromal cell'], dtype=object), array([124, 124, 125, 124, 125, 124, 125, 124]))
dataset composition from each stage of this example bank:

(array([1, 2]), array([435, 560]))




## stage 3

### load the pre-processed dataset ''Limb_Muscle'' 

In [6]:
dataset_stage_3 = "TabulaMuris_Limb_Muscle_10X"
data_path_stage_3 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_3)
adata_stage_3 = sc.read_h5ad(data_path_stage_3)
cell_type_stage_3 = adata_stage_3.obs['cell_type1']
print(adata_stage_3)  
print(np.unique(np.array(cell_type_stage_3), return_counts=True))

AnnData object with n_obs × n_vars = 3409 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['B cell', 'T cell', 'endothelial cell', 'macrophage',
       'mesenchymal stem cell', 'skeletal muscle satellite cell'],
      dtype=object), array([ 398,  269, 1157,  274,  998,  313]))


### fine-tune the CANAL model for the third stage

In [7]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_2, dataset = dataset_stage_3, 
            adata = adata_stage_3, cell_type = cell_type_stage_3, current_stage = 3, is_final_stage = False,   
            ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: AnnData object with n_obs × n_vars = 3409 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['B cell', 'T cell', 'endothelial cell', 'macrophage',
       'mesenchymal stem cell', 'skeletal muscle satellite cell'],
      dtype=object), array([ 398,  269, 1157,  274,  998,  313]))
model constructing begin!

new cell types: ['mesenchymal stem cell', 'skeletal muscle satellite cell']
example bank for experience replay: AnnData object with n_obs × n_vars = 995 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', '

/data/wanh/CANAL/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)


model constructing finished!

label train: [0 1 2 3 4 5 6 7 8 9] 10
label val: [0 1 2 3 4 5 6 7 8 9] 10
  ==  Begin finetuning: | Incrmental stage | Current stage: 3 | CANAL | Dataset: Cross_tissue TabulaMuris_Limb_Muscle_10X  ==


/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


    ==  Epoch: 1 | Classification Loss: 1.269150 | Representation DL Loss: 0.014623  | Accuracy: 67.7291%  ==
    ==  Epoch: 1 | Validation CLS Loss: 3.424092 | Validation Representation DL Loss: 0.033891 | F1 Score: 0.700836  == | Current accuracy: 0.686636  ==
[[ 98   0   0   0   0   0   0   0   0   0]
 [  0  73   1   0   1   0   0   0   0   0]
 [  0   0 254   0   0   0   0   0   0   0]
 [  0   0   0  67   0   1   0   0   0   0]
 [  0   0   0   0  18   0   0   0   0   0]
 [  0   0   0   0   0  26   0   0   0   0]
 [  0   0   0   0   0   0  29   0   0   0]
 [  0   0   0   0   0   0   0  31   0   0]
 [  0   0   2   0   0 200   0   0   0   0]
 [  2  17   0   0   0  46   0   0   2   0]]
Patience: 0 / 10
    ==  Epoch: 2 | Classification Loss: 0.700298 | Representation DL Loss: 0.393118  | Accuracy: 93.5970%  ==
    ==  Epoch: 2 | Validation CLS Loss: 3.925053 | Validation Representation DL Loss: 0.438186 | F1 Score: 0.963895  == | Current accuracy: 0.981567  ==
[[110   0   0   0   0   0 

    ==  Epoch: 13 | Classification Loss: 0.509605 | Representation DL Loss: 0.187941  | Accuracy: 100.0000%  ==
    ==  Epoch: 13 | Validation CLS Loss: 3.599265 | Validation Representation DL Loss: 0.181978 | F1 Score: 0.978821  == | Current accuracy: 0.988479  ==
[[ 82   0   0   0   0   0   0   0   0   0]
 [  0  71   0   0   1   0   0   0   0   0]
 [  0   0 272   0   0   0   0   0   0   0]
 [  0   0   1  68   0   0   0   0   0   0]
 [  0   0   0   0  19   0   0   0   0   0]
 [  0   0   1   0   0  17   0   0   4   0]
 [  0   0   0   0   0   0  22   0   0   0]
 [  0   0   0   0   0   0   0  25   0   0]
 [  0   0   0   1   0   0   0   0 226   0]
 [  0   0   0   0   0   0   0   0   2  56]]
Patience: 1 / 10
    ==  Epoch: 14 | Classification Loss: 0.509407 | Representation DL Loss: 0.181221  | Accuracy: 100.0000%  ==
    ==  Epoch: 14 | Validation CLS Loss: 3.799653 | Validation Representation DL Loss: 0.157986 | F1 Score: 0.976329  == | Current accuracy: 0.983871  ==
[[105   0   0   0   

    ==  Epoch: 25 | Classification Loss: 0.506000 | Representation DL Loss: 0.136285  | Accuracy: 100.0000%  ==
    ==  Epoch: 25 | Validation CLS Loss: 3.553699 | Validation Representation DL Loss: 0.117606 | F1 Score: 0.981745  == | Current accuracy: 0.986175  ==
[[ 91   0   0   0   0   0   0   0   0   0]
 [  0  66   0   0   0   0   0   0   0   0]
 [  0   0 261   0   0   0   0   0   0   0]
 [  0   0   2  84   0   0   0   0   0   0]
 [  0   0   0   0  29   0   0   0   0   0]
 [  0   0   3   0   0  26   0   0   4   0]
 [  0   0   0   0   0   0  24   0   0   0]
 [  0   0   0   0   0   0   0  20   0   0]
 [  0   0   0   2   0   1   0   0 209   0]
 [  0   0   0   0   0   0   0   0   0  46]]
Patience: 8 / 10
    ==  Epoch: 26 | Classification Loss: 0.505666 | Representation DL Loss: 0.139120  | Accuracy: 100.0000%  ==
    ==  Epoch: 26 | Validation CLS Loss: 3.644901 | Validation Representation DL Loss: 0.135973 | F1 Score: 0.984632  == | Current accuracy: 0.990783  ==
[[ 94   0   0   0   

/data/wanh/CANAL/model.py:105: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_i.obs['celltype'] = current_label_dict[i]
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],




example bank after updating:
AnnData object with n_obs × n_vars = 996 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch', 'celltype', 'rank', 'stage'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'


cell type composition of this example bank:
(array(['B cell', 'T cell', 'basal cell', 'endothelial cell',
       'luminal epithelial cell of mammary gland', 'macrophage',
       'mesenchymal stem cell', 'natural killer cell',
       'skeletal muscle satellite cell', 'stromal cell'], dtype=object), array([ 99,  99, 100,  99, 100,  99, 100, 100, 100, 100]))


dataset composition from each stage of this example bank:
(array([1, 2, 3]), array([282, 382, 

## stage 4

### load the pre-processed dataset ''Spleen'' 

In [7]:
dataset_stage_4 = "TabulaMuris_Spleen_10X"
data_path_stage_4 = './data/{}/{}_train.h5ad'.format(experiments, dataset_stage_4)
adata_stage_4 = sc.read_h5ad(data_path_stage_4)
cell_type_stage_4 = adata_stage_4.obs['cell_type1']
print(adata_stage_4)  
print(np.unique(np.array(cell_type_stage_4), return_counts=True))

AnnData object with n_obs × n_vars = 9010 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'
(array(['B cell', 'T cell', 'macrophage', 'natural killer cell'],
      dtype=object), array([6541, 1816,  431,  222]))


### fine-tune the CANAL model for the forth stage

In [8]:
CANAL.train(experiments = experiments, pre_dataset = dataset_stage_3, dataset = dataset_stage_4, 
            adata = adata_stage_4, cell_type = cell_type_stage_4, current_stage = 4, is_final_stage = True,   
            ckpt_dir='./ckpts/', rehearsal_size=1000, SEED = seed)

current data: AnnData object with n_obs × n_vars = 9010 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p'


(array(['B cell', 'T cell', 'macrophage', 'natural killer cell'],
      dtype=object), array([6541, 1816,  431,  222]))
model constructing begin!

new cell types: []
example bank for experience replay: AnnData object with n_obs × n_vars = 996 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_

/data/wanh/CANAL/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)
/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


model constructing finished!

label train: [0 1 2 3 4 5 6 7 8 9] 10
label val: [0 1 2 3 4 5 6 7 8 9] 10
  ==  Begin finetuning: | Final stage | Current stage: 4 | CANAL | Dataset: Cross_tissue TabulaMuris_Spleen_10X ==
    ==  Epoch: 1 | Classification Loss: 0.566440 | Representation DL Loss: 0.005053  | Accuracy: 97.8359%  ==
    ==  Epoch: 1 | Validation CLS Loss: 2.590173 | Validation Representation DL Loss: 0.018307 | F1 Score: 0.969873  == | Current accuracy: 0.979520  ==
[[1292   16    0    1    0    0    0    7    0    0]
 [   6  364    0    1    0    0    0    0    0    0]
 [   0    0   23    0    0    0    0    0    0    0]
 [   4    0    0  108    0    3    0    0    0    0]
 [   0    3    0    0   64    0    0    0    0    0]
 [   0    0    0    0    0   24    0    0    0    0]
 [   0    0    0    0    0    0   33    0    0    0]
 [   0    0    0    0    0    0    0   23    0    0]
 [   0    0    0    0    0    0    0    0   18    0]
 [   0    0    0    0    0    0    0    0

    ==  Epoch: 11 | Classification Loss: 0.508377 | Representation DL Loss: 0.036215  | Accuracy: 99.8374%  ==
    ==  Epoch: 11 | Validation CLS Loss: 2.127766 | Validation Representation DL Loss: 0.039598 | F1 Score: 0.993257  == | Current accuracy: 0.991009  ==
[[1331    8    0    1    0    0    0    0    0    0]
 [   3  360    0    0    3    0    0    0    0    0]
 [   0    0   25    0    0    0    0    0    0    0]
 [   3    0    0  112    0    0    0    0    0    0]
 [   0    0    0    0   58    0    0    0    0    0]
 [   0    0    0    0    0   18    0    0    0    0]
 [   0    0    0    0    0    0   21    0    0    0]
 [   0    0    0    0    0    0    0   27    0    0]
 [   0    0    0    0    0    0    0    0   16    0]
 [   0    0    0    0    0    0    0    0    0   16]]
Patience: 0 / 10
    ==  Epoch: 12 | Classification Loss: 0.508645 | Representation DL Loss: 0.041663  | Accuracy: 99.8374%  ==
    ==  Epoch: 12 | Validation CLS Loss: 3.278699 | Validation Representatio

    ==  Epoch: 21 | Validation CLS Loss: 2.641935 | Validation Representation DL Loss: 0.032302 | F1 Score: 0.991922  == | Current accuracy: 0.987512  ==
[[1322    7    0    2    0    0    0    0    0    0]
 [   8  362    0    0    2    0    0    0    0    0]
 [   0    0   11    0    0    0    0    0    0    0]
 [   5    1    0  113    0    0    0    0    0    0]
 [   0    0    0    0   70    0    0    0    0    0]
 [   0    0    0    0    0   18    0    0    0    0]
 [   0    0    0    0    0    0   25    0    0    0]
 [   0    0    0    0    0    0    0   17    0    0]
 [   0    0    0    0    0    0    0    0   16    0]
 [   0    0    0    0    0    0    0    0    0   23]]
Patience: 10 / 10
    ==  Epoch: 22 | Classification Loss: 0.506135 | Representation DL Loss: 0.036922  | Accuracy: 99.9249%  ==
    ==  Epoch: 22 | Validation CLS Loss: 2.578008 | Validation Representation DL Loss: 0.031964 | F1 Score: 0.996073  == | Current accuracy: 0.990509  ==
[[1295   12    0    0    0    0 

## load the unlabeled 10X test data and apply CANAL to predict cell types

In [8]:
adata_test1_10X=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Lung_10X_test.h5ad")  
print(adata_test1_10X,np.unique(np.array(adata_test1_10X.obs['cell_type1']),return_counts=True))

adata_test2_10X=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Mammary_Gland_10X_test.h5ad")  
print(adata_test2_10X,np.unique(np.array(adata_test2_10X.obs['cell_type1']),return_counts=True))

adata_test3_10X=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Limb_Muscle_10X_test.h5ad")  
print(adata_test3_10X,np.unique(np.array(adata_test3_10X.obs['cell_type1']),return_counts=True))

adata_test4_10X=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Spleen_10X_test.h5ad")  
print(adata_test4_10X,np.unique(np.array(adata_test4_10X.obs['cell_type1']),return_counts=True))

adata_test_10X = sc.AnnData.concatenate(adata_test1_10X,adata_test2_10X,adata_test3_10X,adata_test4_10X)
print(adata_test_10X,np.unique(np.array(adata_test_10X.obs['cell_type1']),return_counts=True))

pred_cell_type_10X = CANAL.predict(adata_predict = adata_test_10X, ckpt_dir = './ckpts/', experiments = experiments,
                               stage_num=4, dataset = dataset_stage_4)

AnnData object with n_obs × n_vars = 500 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p' (array(['B cell', 'T cell', 'endothelial cell', 'macrophage',
       'natural killer cell', 'stromal cell'], dtype=object), array([ 17,  35,  50,  31, 103, 264]))
AnnData object with n_obs × n_vars = 500 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', '

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


AnnData object with n_obs × n_vars = 2000 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm' (array(['B cell', 'T cell', 'basal cell', 'endothelial cell',
       'luminal epithelial cell of mammary gland', 'macrophage',
       'mesenchymal stem cell', 'natural killer cell',
       'skeletal muscle satellite cell', 'stromal cell'], dtype=object), array([503, 403,  51, 251,  48, 120, 138, 111,  41, 334]))
    ==  Begin predicting after 4 fine-tuning stages: | Experiments: Cross_tissue ==
Annotation: ['B cell' 'T cell' 'endothelial cell' 'macrophage' 'natural killer cell'
 'stromal cell' 'basal cell' 'lu

/data/wanh/CANAL/performer_pytorch/performer_pytorch.py:115: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2497.)
  q, r = torch.qr(unstructured_block.cpu(), some = True)


## evaluate the annotation performance

In [9]:
true_celltype_10X = np.array(adata_test_10X.obs['cell_type1']) 
CANAL.evaluation(pred_cell_type=pred_cell_type_10X, true_celltype=true_celltype_10X)

  ==  Predict total accuracy: 0.969000 ==|== F1 Score: 0.971300  ==|==  ARI: 0.938800 == 

Confusion matrix:
[[501   2   0   0   0   0   0   0   0   0]
 [ 10 393   0   0   0   0   0   0   0   0]
 [  0   0  51   0   0   0   0   0   0   0]
 [  0   0   0 251   0   0   0   0   0   0]
 [  0   0   0   0  48   0   0   0   0   0]
 [  2   1   0   0   0 116   1   0   0   0]
 [  0   0   0   0   0   0 134   0   0   4]
 [  0   0   0   0   0   0   0 111   0   0]
 [  0   0   0   0   0   0   0   0  40   1]
 [  0   0   0   2   0   8  31   0   0 293]]


### load the unlabeled SS2 test data and apply CANAL to predict cell types

In [10]:
adata_test_Lung_SS2=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Lung_SS2.h5ad")  
print(adata_test_Lung_SS2,np.unique(np.array(adata_test_Lung_SS2.obs['cell_type1']),return_counts=True))

adata_test_Mammary_Gland_SS2=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Mammary_Gland_SS2.h5ad")  
print(adata_test_Mammary_Gland_SS2,np.unique(np.array(adata_test_Mammary_Gland_SS2.obs['cell_type1']),return_counts=True))

adata_test_Limb_Muscle_SS2=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Limb_Muscle_SS2.h5ad")  
print(adata_test_Limb_Muscle_SS2,np.unique(np.array(adata_test_Limb_Muscle_SS2.obs['cell_type1']),return_counts=True))

adata_test_Spleen_SS2=sc.read_h5ad("/data/wanh/DA/scBERT-master/data/TabulaMuris/TabulaMuris_Spleen_SS2.h5ad")  
print(adata_test_Spleen_SS2,np.unique(np.array(adata_test_Spleen_SS2.obs['cell_type1']),return_counts=True))

adata_test_SS2 = sc.AnnData.concatenate(adata_test_Lung_SS2,adata_test_Mammary_Gland_SS2,adata_test_Limb_Muscle_SS2,adata_test_Spleen_SS2)
print(adata_test_SS2,np.unique(np.array(adata_test_SS2.obs['cell_type1']),return_counts=True))

pred_cell_type_SS2 = CANAL.predict(adata_predict = adata_test_SS2, ckpt_dir = './ckpts/', experiments = experiments,       
                                   stage_num=4, dataset = dataset_stage_4)

AnnData object with n_obs × n_vars = 1263 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'hvg', 'log1p' (array(['B cell', 'T cell', 'endothelial cell', 'natural killer cell',
       'stromal cell'], dtype=object), array([ 57,  53, 693,  37, 423]))
AnnData object with n_obs × n_vars = 2405 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA'

/data/wanh/ENTER/envs/pytorch/lib/python3.7/site-packages/anndata/_core/anndata.py:1785: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  [AnnData(sparse.csr_matrix(a.shape), obs=a.obs) for a in all_adatas],


AnnData object with n_obs × n_vars = 6455 × 1000
    obs: 'cell_type1', 'cell_ontology_id', 'cluster', 'free_annotation', 'donor', 'gender', 'channel', 'region', 'organ', 'tissue_tSNE_1', 'tissue_tSNE_2', 'cell_ontology_class', 'platform', 'dataset_name', 'organism', 'data_type', 'n_genes', 'n_counts', '__libsize__', 'method', 'plate', 'subsetA', 'subsetA_cluster.ids', 'subsetB', 'subsetB_cluster.ids', 'batch'
    var: 'highly_variable', 'means', 'dispersions', 'dispersions_norm' (array(['B cell', 'T cell', 'basal cell', 'endothelial cell',
       'luminal epithelial cell of mammary gland', 'macrophage',
       'mesenchymal stem cell', 'natural killer cell',
       'skeletal muscle satellite cell', 'stromal cell'], dtype=object), array([1425,  440, 1340,  881,  578,   93,  258,   37,  540,  863]))
    ==  Begin predicting after 4 fine-tuning stages: | Experiments: Cross_tissue ==
Annotation: ['B cell' 'T cell' 'endothelial cell' 'macrophage' 'natural killer cell'
 'stromal cell' 'basal

### evaluate the annotation performance

In [11]:
true_celltype_SS2 = np.array(adata_test_SS2.obs['cell_type1']) 
CANAL.evaluation(pred_cell_type=pred_cell_type_SS2, true_celltype=true_celltype_SS2)

  ==  Predict total accuracy: 0.936000 ==|== F1 Score: 0.842000  ==|==  ARI: 0.919200 == 

Confusion matrix:
[[1365    8    0    0    1   49    0    2    0    0]
 [   0  438    0    0    0    1    0    1    0    0]
 [   4    1 1295    3   15    5    0    0    4   13]
 [   0    0    1  877    0    0    1    0    1    1]
 [   0    0    0    0  578    0    0    0    0    0]
 [   0    0    0    0    0   45    0   48    0    0]
 [   0    0    0    0    0    0  250    0    0    8]
 [   0    4    0    0    0    0    0   33    0    0]
 [   0    1    0    2    0    3    3    0  530    1]
 [   0    0    0    8    0    0  224    0    0  631]]
